# Transfer Learning using DenseNet201

## Imports and Preprocess

Import libraries

In [1]:
import numpy as np
import pandas as pd

import pickle
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow import set_random_seed
#from keras.applications.resnet50 import ResNet50, preprocess_input as resnet50_preprocess
from keras.applications.densenet import DenseNet201, preprocess_input as densenet201_preprocess
#from keras.applications.mobilenet_v2 import MobileNetV2
#from keras.applications.nasnet import NASNetMobile
from keras.preprocessing.image import ImageDataGenerator
from keras.utils.np_utils import to_categorical
from keras.callbacks import EarlyStopping, ModelCheckpoint

from src.sampler import train_test_bootstrapper
from src.model_api import getPretrainedModel

Using TensorFlow backend.


Import data

In [2]:
with open('data/derived/data_train.pickle', 'rb') as file:
    data_train = pickle.load(file)
with open('data/derived/labels_train.pickle', 'rb') as file:
    labels_train = pickle.load(file)
print('Shape of data_train:', data_train.shape)
print('Length of labels_train:', len(labels_train))

Shape of data_train: (25361, 224, 224, 3)
Length of labels_train: 25361


Convert labels to integers for model training

In [3]:
label_encoder = LabelEncoder()
labels_train_encoded = label_encoder.fit_transform(labels_train)
print('Original labels: {}'.format(labels_train[:5]))
print('Encoded labels: {}'.format(labels_train_encoded[:5]))
print('Label encoder classes: {}'.format(label_encoder.classes_[:5]))

Original labels: ['w_025911c', 'new_whale', 'new_whale', 'new_whale', 'new_whale']
Encoded labels: [44  0  0  0  0]
Label encoder classes: ['new_whale' 'w_0003639' 'w_0003c59' 'w_0027efa' 'w_00289b1']


Compute sample size and number of classes

In [4]:
sample_size = len(labels_train_encoded)
num_classes = len(set(labels_train_encoded))
print('Sample size:', sample_size)
print('Number of clases:', num_classes)

Sample size: 25361
Number of clases: 5005


## Train DenseNet201

DenseNet Model 1 (Random Seed = 1, assuming no class imbalance)

In [6]:
filepath = 'models/weights-{epoch:02d}-{val_loss:.3f}.hdf5'
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', save_best_only=True, save_weights_only=True,
                             mode='min')
early_stop = EarlyStopping(patience=2, monitor='val_loss')

model = getPretrainedModel(DenseNet201(include_top=False, weights='imagenet', pooling='max'),
                                       num_classes=len(set(labels_train_encoded)),
                                       percent_layers_freeze=0.1)

batch_size = 32
image_gen = ImageDataGenerator(rotation_range=20,
                               width_shift_range=.1,
                               height_shift_range=.1,
                               shear_range=0.5,
                               zoom_range=(0.9, 1.1),
                               fill_mode='constant',
                               horizontal_flip=True)

In [7]:
X_train, X_test, y_train, y_test = train_test_bootstrapper(data_train, labels_train_encoded,
                                                           bootstrapper_size=sample_size,
                                                           class_imbalance=False, random_state=1)
    
X_train = densenet201_preprocess(X_train)
X_test = densenet201_preprocess(X_test)
y_train = to_categorical(y_train, num_classes=num_classes)
y_test = to_categorical(y_test, num_classes=num_classes)
    
image_generator_samples = image_gen.flow(X_train, y_train, batch_size=batch_size, seed=1)
    
np.random.seed(1)
set_random_seed(1)
train_size, epochs = X_train.shape[0], 10
hist = model.fit_generator(image_generator_samples, steps_per_epoch=2*train_size//epochs,
                           validation_data=(X_test, y_test), epochs=epochs, shuffle=True,
                           callbacks=[checkpoint, early_stop])

model.save_weights('models/weights_densenet201_1.hdf5')

Epoch 1/10
5072/5072 [==============================] - 1602s 316ms/step - loss: 5.7216 - acc: 0.3819 - val_loss: 6.3653 - val_acc: 0.3831
Epoch 2/10
5072/5072 [==============================] - 1572s 310ms/step - loss: 5.0150 - acc: 0.3820 - val_loss: 6.2866 - val_acc: 0.3835
Epoch 3/10
5072/5072 [==============================] - 1566s 309ms/step - loss: 4.7255 - acc: 0.3836 - val_loss: 6.4515 - val_acc: 0.3841
Epoch 4/10
5072/5072 [==============================] - 1573s 310ms/step - loss: 4.6083 - acc: 0.3861 - val_loss: 6.5239 - val_acc: 0.3847


DenseNet Model 2 (Random Seed = 2, assuming no class imbalance)

In [5]:
filepath = 'models/weights-{epoch:02d}-{val_loss:.3f}.hdf5'
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', save_best_only=True, save_weights_only=True,
                             mode='min')
early_stop = EarlyStopping(patience=2, monitor='val_loss')

model = getPretrainedModel(DenseNet201(include_top=False, weights='imagenet', pooling='max'),
                           num_classes=len(set(labels_train_encoded)),
                           percent_layers_freeze=0.1)

batch_size = 32
image_gen = ImageDataGenerator(rotation_range=20,
                               width_shift_range=.1,
                               height_shift_range=.1,
                               shear_range=0.5,
                               zoom_range=(0.9, 1.1),
                               fill_mode='constant',
                               horizontal_flip=True)

In [6]:
X_train, X_test, y_train, y_test = train_test_bootstrapper(data_train, labels_train_encoded,
                                                           bootstrapper_size=sample_size,
                                                           class_imbalance=False, random_state=2)
    
X_train = densenet201_preprocess(X_train)
X_test = densenet201_preprocess(X_test)
y_train = to_categorical(y_train, num_classes=num_classes)
y_test = to_categorical(y_test, num_classes=num_classes)
    
image_generator_samples = image_gen.flow(X_train, y_train, batch_size=batch_size, seed=2)
    
np.random.seed(2)
set_random_seed(2)
train_size, epochs = X_train.shape[0], 10
hist = model.fit_generator(image_generator_samples, steps_per_epoch=2*train_size//epochs,
                           validation_data=(X_test, y_test), epochs=epochs, shuffle=True,
                           callbacks=[checkpoint, early_stop])

model.save_weights('models/weights_densenet201_2.hdf5')

Epoch 1/10
5072/5072 [==============================] - 1579s 311ms/step - loss: 5.6498 - acc: 0.3828 - val_loss: 6.4239 - val_acc: 0.3775
Epoch 2/10
5072/5072 [==============================] - 1579s 311ms/step - loss: 4.9054 - acc: 0.3845 - val_loss: 6.1716 - val_acc: 0.3780
Epoch 3/10
5072/5072 [==============================] - 1581s 312ms/step - loss: 4.6185 - acc: 0.3863 - val_loss: 6.5170 - val_acc: 0.3807
Epoch 4/10
5072/5072 [==============================] - 1573s 310ms/step - loss: 4.5008 - acc: 0.3881 - val_loss: 6.1504 - val_acc: 0.3793
Epoch 5/10
5072/5072 [==============================] - 1580s 312ms/step - loss: 4.4308 - acc: 0.3892 - val_loss: 6.2744 - val_acc: 0.3816
Epoch 6/10
5072/5072 [==============================] - 1568s 309ms/step - loss: 4.3865 - acc: 0.3901 - val_loss: 6.1912 - val_acc: 0.3826


DenseNet Model 3 (Random Seed = 1, considering class imbalance)

In [5]:
filepath = 'models/weights-{epoch:02d}-{val_loss:.3f}.hdf5'
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', save_best_only=True, save_weights_only=True,
                             mode='min')
early_stop = EarlyStopping(patience=2, monitor='val_loss')

model = getPretrainedModel(DenseNet201(include_top=False, weights='imagenet', pooling='max'),
                           num_classes=len(set(labels_train_encoded)),
                           percent_layers_freeze=0.1)

batch_size = 32
image_gen = ImageDataGenerator(rotation_range=20,
                               width_shift_range=.1,
                               height_shift_range=.1,
                               shear_range=0.5,
                               zoom_range=(0.9, 1.1),
                               fill_mode='constant',
                               horizontal_flip=True)

In [6]:
X_train, X_test, y_train, y_test = train_test_bootstrapper(data_train, labels_train_encoded,
                                                           bootstrapper_size=sample_size,
                                                           class_imbalance=True, random_state=1)
    
X_train = densenet201_preprocess(X_train)
X_test = densenet201_preprocess(X_test)
y_train = to_categorical(y_train, num_classes=num_classes)
y_test = to_categorical(y_test, num_classes=num_classes)
    
image_generator_samples = image_gen.flow(X_train, y_train, batch_size=batch_size, seed=1)
    
np.random.seed(1)
set_random_seed(1)
train_size, epochs = X_train.shape[0], 10
hist = model.fit_generator(image_generator_samples, steps_per_epoch=2*train_size//epochs,
                           validation_data=(X_test, y_test), epochs=epochs, shuffle=True,
                           callbacks=[checkpoint, early_stop])

model.save_weights('models/weights_densenet201_3.hdf5')

Epoch 1/10
5072/5072 [==============================] - 1583s 312ms/step - loss: 7.1346 - acc: 0.1885 - val_loss: 4.7227 - val_acc: 0.5752
Epoch 2/10
5072/5072 [==============================] - 1553s 306ms/step - loss: 6.5429 - acc: 0.1893 - val_loss: 4.3267 - val_acc: 0.5752
Epoch 3/10
5072/5072 [==============================] - 1545s 305ms/step - loss: 6.3507 - acc: 0.1901 - val_loss: 4.2276 - val_acc: 0.5759
Epoch 4/10
5072/5072 [==============================] - 1531s 302ms/step - loss: 6.2705 - acc: 0.1914 - val_loss: 4.2252 - val_acc: 0.5753
Epoch 5/10
5072/5072 [==============================] - 1575s 311ms/step - loss: 6.2239 - acc: 0.1911 - val_loss: 4.4160 - val_acc: 0.5749
Epoch 6/10
5072/5072 [==============================] - 1575s 311ms/step - loss: 6.1885 - acc: 0.1913 - val_loss: 4.1757 - val_acc: 0.5760
Epoch 7/10
5072/5072 [==============================] - 1577s 311ms/step - loss: 6.1608 - acc: 0.1920 - val_loss: 4.2577 - val_acc: 0.5757
Epoch 8/10
5072/5072 [=====

DenseNet Model 4 (Random Seed = 2, considering class imbalance)

In [7]:
filepath = 'models/weights-{epoch:02d}-{val_loss:.3f}.hdf5'
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', save_best_only=True, save_weights_only=True,
                             mode='min')
early_stop = EarlyStopping(patience=2, monitor='val_loss')

model = getPretrainedModel(DenseNet201(include_top=False, weights='imagenet', pooling='max'),
                           num_classes=len(set(labels_train_encoded)),
                           percent_layers_freeze=0.1)

batch_size = 32
image_gen = ImageDataGenerator(rotation_range=20,
                               width_shift_range=.1,
                               height_shift_range=.1,
                               shear_range=0.5,
                               zoom_range=(0.9, 1.1),
                               fill_mode='constant',
                               horizontal_flip=True)

In [8]:
X_train, X_test, y_train, y_test = train_test_bootstrapper(data_train, labels_train_encoded,
                                                           bootstrapper_size=sample_size,
                                                           class_imbalance=True, random_state=2)
    
X_train = densenet201_preprocess(X_train)
X_test = densenet201_preprocess(X_test)
y_train = to_categorical(y_train, num_classes=num_classes)
y_test = to_categorical(y_test, num_classes=num_classes)
    
image_generator_samples = image_gen.flow(X_train, y_train, batch_size=batch_size, seed=2)
    
np.random.seed(2)
set_random_seed(2)
train_size, epochs = X_train.shape[0], 10
hist = model.fit_generator(image_generator_samples, steps_per_epoch=2*train_size//epochs,
                           validation_data=(X_test, y_test), epochs=epochs, shuffle=True,
                           callbacks=[checkpoint, early_stop])

model.save_weights('models/weights_densenet201_4.hdf5')

Epoch 1/10
5072/5072 [==============================] - 1606s 317ms/step - loss: 7.1278 - acc: 0.1882 - val_loss: 4.7859 - val_acc: 0.5702
Epoch 2/10
5072/5072 [==============================] - 1586s 313ms/step - loss: 6.5771 - acc: 0.1893 - val_loss: 4.5678 - val_acc: 0.5702
Epoch 3/10
5072/5072 [==============================] - 1588s 313ms/step - loss: 6.4106 - acc: 0.1900 - val_loss: 4.4918 - val_acc: 0.5700
Epoch 4/10
5072/5072 [==============================] - 1584s 312ms/step - loss: 6.3385 - acc: 0.1905 - val_loss: 4.5803 - val_acc: 0.5697
Epoch 5/10
5072/5072 [==============================] - 1587s 313ms/step - loss: 6.2852 - acc: 0.1914 - val_loss: 4.4046 - val_acc: 0.5703
Epoch 6/10
5072/5072 [==============================] - 1584s 312ms/step - loss: 6.2574 - acc: 0.1916 - val_loss: 4.6082 - val_acc: 0.5696
Epoch 7/10
5072/5072 [==============================] - 1584s 312ms/step - loss: 6.2275 - acc: 0.1916 - val_loss: 4.4326 - val_acc: 0.5644
